In [29]:
from function import *
import numpy as np
import networkx as nx
from collections import defaultdict
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import matplotlib.pyplot as plt

#### LOAD THE DATA

In [30]:
# Using Random Data
# N = 100000                # Number of data
# d = 2                     # Number of features
# precision = 2             # Control precision bit of the output
# dim = d * (2 * precision) # Size of the QUBO matrix
# degree = 2                # Degree of the polynomial regression

# # Generate Random Data
# X = np.random.rand(N, d)
# Y = np.random.rand(N)

# Using Dataset
dataset = 'dataset/processed dataset/dataset_2b.csv'
df = pd.read_csv(dataset)

X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values
d = X.shape[1]
precision = 3
dim = d * (2 * precision)
degree = 2

# Train-Test data splitting
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# If you want to perform polynomial regression
X_train, d, dim = polynomialForm(X_train, d, dim, precision, degree)
X_test, _, _ = polynomialForm(X_test, d, dim, precision, degree) 

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(614, 45)
(154, 45)
(614,)
(154,)


#### COMPUTING SOME IMPORTANT VARIABLES
Initiating variables that required to creating QUBO Matrix

In [31]:
XtX = np.matmul(X_train.T, X_train) # Covariance matrix
XtY = np.matmul(X_train.T, Y_train) # Relation between Features and Y

print(XtX)
print(XtY)

[[614.         241.63888889 332.75       ... 281.640625   215.55
  251.56      ]
 [241.63888889 147.66589506  77.55787037 ... 111.40581597  85.33541667
   98.82      ]
 [332.75        77.55787037 235.45138889 ... 151.79947917 116.14166667
  136.47      ]
 ...
 [281.640625   111.40581597 151.79947917 ... 225.57739258 145.7296875
  122.45875   ]
 [215.55        85.33541667 116.14166667 ... 145.7296875  122.45875
  128.598     ]
 [251.56        98.82       136.47       ... 122.45875    128.598
  178.7344    ]]
[223.42068408 117.34788431  89.17820271 112.41229656  81.12152497
 179.9461352  113.0666128  142.65381767 127.10697549  77.40228339
  31.66134502  53.25916129  26.86864221 111.42236736  59.28866794
  74.56569542  66.78236077  50.53079271  49.75790338  48.49117715
  53.64898106  45.29423647  57.36034709  50.74275967  70.72941733
  36.31261249  94.97960833  57.48856656  71.0860394   63.87205571
  54.43876537  37.64697609  40.8048997   52.8921694   46.2247419
 179.9461352   90.68578867

#### FIXSTAR AMPLIFY SIMULATED ANNEALER

In [32]:
model = generateQuboMatrix_fixstar(XtX, XtY, dim, precision, d)
distributions_fixstar, weights_fixstar = solve_fixstar(model, dim, precision, d)

In [33]:
Y_train_pred_fixstar = np.dot(X_train, weights_fixstar)
Y_test_pred_fixstar = np.dot(X_test, weights_fixstar)

# Evaluate on training data
train_mse_fixstar = mean_squared_error(Y_train, Y_train_pred_fixstar)
train_mae_fixstar = mean_absolute_error(Y_train, Y_train_pred_fixstar)
train_r2_fixstar = r2_score(Y_train, Y_train_pred_fixstar) 

# Evaluate on test data
test_mse_fixstar = mean_squared_error(Y_test, Y_test_pred_fixstar)
test_mae_fixstar = mean_absolute_error(Y_test, Y_test_pred_fixstar)
test_r2_fixstar = r2_score(Y_test, Y_test_pred_fixstar)

# Print the metrics
print("Fixstar Training Evaluation:")
print(f"MSE: {train_mse_fixstar:.5f}, MAE: {train_mae_fixstar:.5f}, R²: {train_r2_fixstar:.5f}")    

print("\nFixstar Testing Evaluation:")
print(f"MSE: {test_mse_fixstar:.5f}, MAE: {test_mae_fixstar:.5f}, R²: {test_r2_fixstar:.5f}")  

Fixstar Training Evaluation:
MSE: 0.00651, MAE: 0.05705, R²: 0.89984

Fixstar Testing Evaluation:
MSE: 0.00672, MAE: 0.05815, R²: 0.89997


#### NEAL SIMULATED ANNEALER

In [34]:
Q = generateQuboMatrix_neal(XtX, XtY, precision, d)
sampleset = sampling(Q)
distributions_neal, weights_neal = solve_neal(sampleset, dim, precision, d)

In [35]:
Y_train_pred_neal = np.dot(X_train, weights_neal)
Y_test_pred_neal = np.dot(X_test, weights_neal)

# Evaluate on training data
train_mse_neal = mean_squared_error(Y_train, Y_train_pred_neal)
train_mae_neal = mean_absolute_error(Y_train, Y_train_pred_neal)
train_r2_neal = r2_score(Y_train, Y_train_pred_neal)

# Evaluate on test data
test_mse_neal = mean_squared_error(Y_test, Y_test_pred_neal)
test_mae_neal = mean_absolute_error(Y_test, Y_test_pred_neal)
test_r2_neal = r2_score(Y_test, Y_test_pred_neal)

# Print the metrics
print("Neal Training Evaluation:")
print(f"MSE: {train_mse_neal:.5f}, MAE: {train_mae_neal:.5f}, R²: {train_r2_neal:.5f}")

print("\nNeal Testing Evaluation:")
print(f"MSE: {test_mse_neal:.5f}, MAE: {test_mae_neal:.5f}, R²: {test_r2_neal:.5f}")

Neal Training Evaluation:
MSE: 0.03299, MAE: 0.14187, R²: 0.49275

Neal Testing Evaluation:
MSE: 0.03353, MAE: 0.14552, R²: 0.50108


#### SCIKIT LEARN (TRADISIONAL WAY)

In [36]:
import time
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse

start_time = time.time() 

clf = LinearRegression()
clf.fit(X_train, Y_train)
print("Coefficients:", clf.coef_)

Y_train_pred_sklearn = clf.predict(X_train)
Y_test_pred_sklearn = clf.predict(X_test)

# Evaluate on training data
train_mse_sklearn = mean_squared_error(Y_train, Y_train_pred_sklearn)
train_mae_sklearn = mean_absolute_error(Y_train, Y_train_pred_sklearn)
train_r2_sklearn = r2_score(Y_train, Y_train_pred_sklearn) 

# Evaluate on test data
test_mse_sklearn = mean_squared_error(Y_test, Y_test_pred_sklearn)
test_mae_sklearn = mean_absolute_error(Y_test, Y_test_pred_sklearn)
test_r2_sklearn = r2_score(Y_test, Y_test_pred_sklearn)

end_time = time.time()  
execution_time = end_time - start_time

print(f"Execution time: {execution_time:.5f} seconds\n")

# Print the metrics
print("Fixstar Training Evaluation:")
print(f"MSE: {train_mse_sklearn:.5f}, MAE: {train_mae_sklearn:.5f}, R²: {train_r2_sklearn:.5f}")    

print("\nFixstar Testing Evaluation:")
print(f"MSE: {test_mse_sklearn:.5f}, MAE: {test_mae_sklearn:.5f}, R²: {test_r2_sklearn:.5f}")  

Coefficients: [ 3.30618190e-11  8.76536516e+02  6.40329815e+01  2.50326720e+02
 -1.09321251e+02  5.07066090e+01 -7.73737566e-02  3.90531126e-01
  9.36590398e-02 -8.44354983e+02 -4.86101504e+02 -5.25650374e+02
 -4.45102665e+01  5.39421064e+02  1.03527960e-02  6.55164409e-02
 -2.26341634e-02  8.55916355e+01 -2.01751176e+02  2.85269313e+02
  1.22844065e+02  9.53948460e-03 -1.33373746e-01 -2.19234772e-02
 -1.04509111e+02 -1.32088544e+02  3.00439110e+02 -1.40770851e-02
  8.41593731e-02 -1.05309617e-02  4.58801007e+02 -5.86146420e+01
  6.47399064e-03 -1.56504259e-01 -2.27433782e-04  5.07066090e+01
  1.16211625e-02 -1.48352875e-01  1.29050950e-02  5.28454434e-02
  1.47238945e-02  1.95238696e-02 -2.95049691e-02 -6.15371451e-02
 -4.42769872e-02]
Execution time: 0.05687 seconds

Fixstar Training Evaluation:
MSE: 0.00175, MAE: 0.02814, R²: 0.97307

Fixstar Testing Evaluation:
MSE: 0.00216, MAE: 0.03207, R²: 0.96785


#### VISUALIZATION OF GRAPH

In [37]:
# # Initialize the graph and add all nodes first
# G = nx.Graph()
# nodes = set()
# for (i, j) in Q.keys():
#     nodes.add(i)
#     nodes.add(j)
# G.add_nodes_from(nodes)

# # Now add edges and node attributes
# for (i, j), weight in Q.items():
#     if weight != 0 and i <= j:  # Avoid duplicates and zero-weights
#         if i == j:
#             G.nodes[i]['bias'] = weight  # Now safe because node exists
#         else:
#             G.add_edge(i, j, weight=weight)

# # Visualization
# plt.figure(figsize=(12, 10))
# pos = nx.circular_layout(G)

# # Draw nodes and edges
# nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=800)
# nx.draw_networkx_labels(G, pos, font_size=12)

# # Scale edge widths by absolute weight
# max_weight = max(abs(d['weight']) for _, _, d in G.edges(data=True))
# edge_widths = [0.1 + 2 * abs(G[u][v]['weight']) / max_weight for u, v in G.edges()]
# nx.draw_networkx_edges(G, pos, width=edge_widths, edge_color='gray')

# # Add edge labels (rounded)
# edge_labels = {(u, v): f"{d['weight']:.1f}" for u, v, d in G.edges(data=True)}
# nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

# # Add node biases
# for node, (x, y) in pos.items():
#     bias = G.nodes[node].get('bias', 0)
#     plt.text(x, y-0.1, f"bias={bias:.1f}", ha='center', fontsize=8, color='red')

# plt.title("QUBO Graph (Edge Width ∝ |Weight|)", fontsize=14)
# plt.axis('off')
# plt.show()

In [38]:
# pos = nx.spring_layout(G, k=0.5, weight='weight', iterations=100)
# nx.draw(G, pos, with_labels=True, 
#        node_color='lightblue', node_size=800,
#        width=[0.1 + 2*abs(G[u][v]['weight'])/max_weight for u,v in G.edges()],
#        edge_color='gray')

In [39]:
# from mpl_toolkits.mplot3d import Axes3D

# pos = nx.spring_layout(G, dim=3, weight='weight')
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# for u, v in G.edges():
#     ax.plot([pos[u][0], pos[v][0]], 
#             [pos[u][1], pos[v][1]], 
#             [pos[u][2], pos[v][2]], 
#             'gray')
# ax.scatter(*np.array(list(pos.values())).T, s=100, c='lightblue')
# plt.show()

#### FINAL EVALUATION

In [40]:
data = {
    "Model": ["Neal", "Neal", "Fixstar", "Fixstar", "Sklearn", "Sklearn"],
    "Dataset": ["Train", "Test", "Train", "Test", "Train", "Test"],
    "MSE": [
        train_mse_neal, test_mse_neal,
        train_mse_fixstar, test_mse_fixstar,
        train_mse_sklearn, test_mse_sklearn
    ],
    "MAE": [
        train_mae_neal, test_mae_neal,
        train_mae_fixstar, test_mae_fixstar,
        train_mae_sklearn, test_mae_sklearn
    ],
    "R2 Score": [
        train_r2_neal, test_r2_neal,
        train_r2_fixstar, test_r2_fixstar,
        train_r2_sklearn, test_r2_sklearn
    ]
}

df_eval = pd.DataFrame(data)
df_eval

,Model,Dataset,MSE,MAE,R2 Score
0,Neal,Train,0.032989,0.141873,0.492747
1,Neal,Test,0.033532,0.145523,0.501079
2,Fixstar,Train,0.006514,0.057053,0.899844
3,Fixstar,Test,0.006723,0.058149,0.899973
4,Sklearn,Train,0.001751,0.028138,0.973070
5,Sklearn,Test,0.002161,0.032073,0.967849
